In [1]:
import sys
sys.path.append('/home/prasse/work/Projekte/AEye/reading-comprehension/')
from rf_baseline  import rf_feature_extraction as rf_feature_extraction
import datetime
from typing import Tuple
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import json
import random
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


/usr/local/apps/Anaconda3/2020.11/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
split_criterion = 'subj'
#split_criterion = 'book-page'
#split_criterion = 'book'
label = 'acc_level'
#label = 'native'
#label = 'subj_acc_level'
#label = 'difficulty'
normalize_flag = True
grid_search = False
use_gaze_entropy_features = True
flag_use_page_setting = False
if flag_use_page_setting:
    # page setting
    input_window = 'page'
    save_dir     = '/home/prasse/work/Projekte/AEye/reading-comprehension/results_surprisal_page/'
    suffix       = ''
else:
    # window setting
    input_window = 'window'
    save_dir     = '/home/prasse/work/Projekte/AEye/reading-comprehension/results_surprisal_21_fix/'
    suffix       = '_window'
num_words = 150

param_grid = { 
    'n_estimators': [100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,8,16, None],
    #'criterion' :['gini', 'entropy'],
    'n_jobs': [-1]
}


In [3]:
spit_criterions = ['subj','book-page','book']
labels = ['acc_level','native','difficulty','subj_acc_level']

model_name = 'rf_flatten'
model_name = 'rf_mean'
model_name = 'rf_features'
axis = 1
n_estimators = 100
if model_name == 'rf_flatten':      
    feature_names = []
    for word_id in range(num_words):
        for i in range(len(feature_names_per_word)):
            feature_names.append('word ' + str(word_id+1) + ' '+ feature_names_per_word[i])
elif model_name == 'rf_mean':
    if axis == 2:
        feature_names = ['word ' + str(i+1) for i in range(num_words)]
    elif axis == 1:
        feature_names = feature_names_per_word

feature_names_per_word = [      'ff',
                                'tf',
                                'wfc_ff_nomarlized',
                                'wfc_tf_normalized',
                                'sc_ff_normalized',
                                'sc_tf_normalized',
                                'ic_ff_normalized',
                                'ic_tf_normalized',
                                'regression',
                                'num_regressions',
                                'num_progressions',
                                'surprisal',
                                'word_len',
                                'dependencies_right',
                                'dependencies_left',
                                'dependency_distance']


for entity in rf_feature_extraction.entity_list:
    feature_names_per_word.append('is_entity_' + entity)

for pos in rf_feature_extraction.pos_list:
    feature_names_per_word.append('is_pos_' + pos)


unique_content_word_list = list(np.unique(list(rf_feature_extraction.content_word_dict.values())))
for content_word in unique_content_word_list:
    feature_names_per_word.append('is_content_word_' + content_word)


unique_reduced_pos_list = list(np.unique(list(rf_feature_extraction.reduced_pos_dict.values())))
for r_pos in unique_reduced_pos_list:
    feature_names_per_word.append('is_reduced_pos_' + r_pos)

feature_names_per_word += ['x_mean','y_mean']

feature_string = '_text_sequence_'
feature_string = '_text_sequence_with_paul_david_cluster_'
feature_string = '_text_sequence_with_paul_david_cluster_41_feature_'
feature_string = '_text_sequence_with_paul_david_cluster_41_feature_'
feature_string = '_full_feature_set_'



for split_criterion in spit_criterions:
    for label in labels:
        save_path=f'{save_dir}{model_name}_{split_criterion}_text_sequence_{label}_{input_window}.csv'

        SB_SAT_PATH = f'/raid/projects/reich3/reading-comprehension/paper_splits/{input_window}/{split_criterion}/'
        split_criterion_dict = {
            'subj': 0,
            'book': 1,
        }
        with open('/raid/projects/reich3/reading-comprehension/npy_files/labels_dict.json') as fp:
            label_dict = json.load(fp)

        if split_criterion == 'book':
            num_folds = 4
        else:
            num_folds = 5
        pd_init = pd.DataFrame(
            columns=[
                'ahn_baseline',
                'fold0_auc', 'fold1_auc', 'fold2_auc', 'fold3_auc', 'fold4_auc',
                'fold0_tpr', 'fold1_tpr', 'fold2_tpr', 'fold3_tpr', 'fold4_tpr',
                'fold0_fpr', 'fold1_fpr', 'fold2_fpr', 'fold3_fpr', 'fold4_fpr',
                'fold0_y_pred', 'fold1_y_pred', 'fold2_y_pred', 'fold3_y_pred', 'fold4_y_pred',
                'fold0_y_test', 'fold1_y_test', 'fold2_y_test', 'fold3_y_test', 'fold4_y_test',
            'avg_auc', 'std_auc']
        )
        pd_init['ahn_baseline'] = [model_name]

        if input_window == 'window':
            input_shape = (21, )
        elif input_window == 'page':
            input_shape = (150, 11)




        feature_importances = []
        for fold in range(num_folds):
            np.random.seed(fold)
            random.seed(fold)
            X_train_path = os.path.join(SB_SAT_PATH, f'X_train_{split_criterion}{feature_string}{fold}{suffix}.npy')
            y_train_path = os.path.join(SB_SAT_PATH, f'y_train_{split_criterion}{feature_string}{fold}{suffix}.npy')
            x_train_all, y_train_all = np.load(X_train_path), np.load(y_train_path, allow_pickle=True)
            if normalize_flag:
                scaler = MinMaxScaler()
                x_train_all = scaler.fit_transform(x_train_all.reshape(-1, x_train_all.shape[-1])).reshape(x_train_all.shape)
            
            
            if grid_search:
                #splitkeys = np.array(sorted(list(set(y_train_all[:, split_criterion_dict[split_criterion]]))))
                outer_cv = KFold(n_splits=3, shuffle=True, random_state=fold)
                y_train_label = np.array(y_train_all[:, label_dict[label]], dtype=int)    
                cv_splits = outer_cv.split(x_train_all,y_train_label)#splitkeys)
                
                if model_name == 'rf_flatten':
                    x_train = np.reshape(x_train_all,[x_train_all.shape[0],x_train_all.shape[1] * x_train_all.shape[2]])
                elif model_name == 'rf_mean':
                    x_train = np.mean(x_train_all,axis=axis)
                elif model_name == 'rf_features':
                    x_train, feature_names = rf_feature_extraction.get_rf_features_for_word_features(
                        x_train_all,feature_names_per_word,disable = False,
                        use_gaze_entropy_features = use_gaze_entropy_features)
            else:                
                if split_criterion != 'book':
                    outer_cv = KFold(n_splits=4, shuffle=True, random_state=fold)
                else:
                    outer_cv = KFold(n_splits=3, shuffle=True, random_state=fold)
                
                if split_criterion != 'book-page':
                    splitkeys = np.array(sorted(list(set(y_train_all[:, split_criterion_dict[split_criterion]]))))
                else:
                    splitkeys = y_train_all[:, label_dict[label]]

                for train_idx, val_idx in outer_cv.split(splitkeys):
                    break

                if split_criterion != 'book-page':
                    N_train_sub = splitkeys[train_idx]
                    N_test_sub = splitkeys[val_idx]

                    x_train = x_train_all[np.isin(y_train_all[:, split_criterion_dict[split_criterion]], N_train_sub)]
                    x_val = x_train_all[np.isin(y_train_all[:, split_criterion_dict[split_criterion]], N_test_sub)]
                    y_train = y_train_all[np.isin(y_train_all[:, split_criterion_dict[split_criterion]], N_train_sub)]
                    y_val = y_train_all[np.isin(y_train_all[:, split_criterion_dict[split_criterion]], N_test_sub)]
                else:
                    x_train = x_train_all[train_idx]
                    y_train = y_train_all[train_idx]
                    x_val = x_train_all[val_idx]
                    y_val = y_train_all[val_idx]
                    y_train_all[val_idx]
                y_train = np.array(y_train[:, label_dict[label]], dtype=int)
                y_val = np.array(y_val[:, label_dict[label]], dtype=int)



                if model_name == 'rf_flatten':
                    x_train = np.reshape(x_train,[x_train.shape[0],x_train.shape[1] * x_train.shape[2]])
                    x_val   = np.reshape(x_val,[x_val.shape[0],x_val.shape[1] * x_val.shape[2]])
                elif model_name == 'rf_mean':
                    x_train = np.mean(x_train,axis=axis)
                    x_val =   np.mean(x_val,axis=axis)
                elif model_name == 'rf_features':
                    x_train, feature_names = rf_feature_extraction.get_rf_features_for_word_features(
                                x_train,feature_names_per_word,disable = False,
                                use_gaze_entropy_features = use_gaze_entropy_features)
                    x_val, _ = rf_feature_extraction.get_rf_features_for_word_features(
                                x_val,feature_names_per_word,disable = False,
                                use_gaze_entropy_features = use_gaze_entropy_features)
            
            # fit RF
            if grid_search:
                model = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=outer_cv,
                                    verbose = 1)
                model.fit(x_train,y_train_label)
                model = model.best_estimator_
            else:
                model = RandomForestClassifier(n_jobs = -1,n_estimators=n_estimators)
                x_train = np.vstack([x_train,x_val])
                y_train = np.concatenate([y_train,y_val])
                model.fit(
                    x_train,
                    y_train)
            feature_importances.append(model.feature_importances_)

            X_test_path = os.path.join(SB_SAT_PATH, f'X_test_{split_criterion}{feature_string}{fold}{suffix}.npy')
            y_test_path = os.path.join(SB_SAT_PATH, f'y_test_{split_criterion}{feature_string}{fold}{suffix}.npy')
            x_test, y_test = np.load(X_test_path), np.load(y_test_path, allow_pickle=True)
            if normalize_flag:
                x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)
            y_test = np.array(y_test[:, label_dict[label]], dtype=int)


            if model_name == 'rf_flatten':
                x_test = np.reshape(x_test,[x_test.shape[0],x_test.shape[1] * x_test.shape[2]])
            elif model_name == 'rf_mean':
                x_test = np.mean(x_test,axis=axis)
            elif model_name == 'rf_features':
                x_test, _ = rf_feature_extraction.get_rf_features_for_word_features(
                            x_test,feature_names_per_word,disable = False,
                            use_gaze_entropy_features = use_gaze_entropy_features)


            y_pred = model.predict_proba(x_test)[:,1]
            try:
                fpr, tpr, _ = metrics.roc_curve(
                    y_test,
                    y_pred,
                    pos_label=1,
                )
                auc = metrics.auc(fpr, tpr)
                print(auc)
                pd_init[f'fold{fold}_auc'] = auc
                pd_init[f'fold{fold}_tpr'] = [tpr]
                pd_init[f'fold{fold}_fpr'] = [fpr]
                pd_init[f'fold{fold}_y_test'] = [y_test]
                pd_init[f'fold{fold}_y_pred'] = [y_pred]
            except:
                try:
                    fpr, tpr, _ = metrics.roc_curve(
                        y_test,
                        y_pred,
                        pos_label=1,
                    )
                    auc = metrics.auc(fpr, tpr)
                    print(auc)
                    pd_init[f'fold{fold}_auc'] = auc
                    pd_init[f'fold{fold}_tpr'] = [tpr]
                    pd_init[f'fold{fold}_fpr'] = [fpr]
                    pd_init[f'fold{fold}_y_test'] = y_test
                    pd_init[f'fold{fold}_y_pred'] = y_pred
                except:
                    print(allo)

        pd_init['avg_auc'] = 0
        for i in range(num_folds):
            pd_init['avg_auc'] += pd_init[f'fold{i}_auc']
        pd_init['avg_auc']  /= num_folds

        pd_init['std_auc'] = 0
        for i in range(0, num_folds):
            pd_init['std_auc'] +=  (pd_init[f'fold{i}_auc'] - pd_init['avg_auc'])**2
        pd_init['std_auc'] = (pd_init['std_auc']/num_folds)**(1/2)
        pd_init.to_csv(save_path, index=None)

100%|██████████| 2579/2579 [00:11<00:00, 215.49it/s]


0.5442413028616346


100%|██████████| 2727/2727 [00:12<00:00, 216.03it/s]


0.5008132307959303


100%|██████████| 2745/2745 [00:12<00:00, 215.55it/s]


...

0.6097032923888048


100%|██████████| 3064/3064 [00:14<00:00, 217.44it/s]


0.6070441985967068


100%|██████████| 3596/3596 [00:16<00:00, 221.34it/s]


0.5996594420762471
